In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
!pip install stanza


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 33.2 MB/s eta 0:00:00


In [17]:
import stanza
stanza.download('ta')  # Download the Tamil language model
nlp = stanza.Pipeline('ta', processors='tokenize,pos', use_gpu=False)  # Pipeline for tokenization and POS tagging


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ta (Tamil) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ta/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ta (Tamil):
| Processor | Package      |
----------------------------
| tokenize  | ttb          |
| mwt       | ttb          |
| pos       | ttb_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they ar

In [19]:
# Define the sentence you want to process
corrected_sentence = "உங்கள் வாக்கியம் இங்கே."  # Replace with your actual sentence

# The rest of your code remains the same
doc = nlp(corrected_sentence)  # Process the sentence
for sent in doc.sentences:
    for word in sent.words:
        print(f"{word.text}\t{word.upos}")  # Print word and POS tag

உங்கள்	PRON
வாக்கியம்	NOUN
இங்கே.	PUNCT


In [20]:
def check_and_correct_grammar(sentence):
    doc = nlp(sentence)  # Process sentence
    errors = []
    corrected_words = sentence.split()

    # Extract words and POS tags
    words = []
    pos_tags = []
    for sent in doc.sentences:
        for word in sent.words:
            words.append(word.text)
            pos_tags.append(word.upos)

    # Rule 1: Subject-Object-Verb Order
    if 'PRON' in pos_tags and 'NOUN' in pos_tags and 'VERB' in pos_tags:
        pron_index = pos_tags.index('PRON')
        noun_index = pos_tags.index('NOUN')
        verb_index = pos_tags.index('VERB')
        if not (pron_index < noun_index < verb_index):
            errors.append("Error: The sentence should follow Subject-Object-Verb (SOV) order.")
            # Correct the word order while preserving other parts of the sentence
            corrected_words = [words[pron_index], words[noun_index], words[verb_index]] + \
                               [word for i, word in enumerate(words) if i not in (pron_index, noun_index, verb_index)]

    # Rule 2: Adjective-Noun Order
    if 'ADJ' in pos_tags and 'NOUN' in pos_tags:
        for i, (tag, word) in enumerate(zip(pos_tags, words)):
            if tag == 'ADJ':
                adj_index = i
                for j in range(adj_index + 1, len(pos_tags)):
                    if pos_tags[j] == 'NOUN':
                        if adj_index > j:
                            errors.append("Error: Adjectives should precede the noun they modify.")
                            words[adj_index], words[j] = words[j], words[adj_index]
                        break

    # Rule 3: Plural Agreement
    if 'PRON' in pos_tags and 'VERB' in pos_tags:
        pron_index = pos_tags.index('PRON')
        verb_index = pos_tags.index('VERB')
        pron_word = words[pron_index]
        verb_word = words[verb_index]
        if pron_word.endswith("ள்") and not verb_word.endswith("ோம்"):
            errors.append("Error: Plural pronoun should match plural verb form.")
            if "ேன்" in verb_word:
                corrected_words[verb_index] = verb_word.replace("ேன்", "ோம்")
            else:
                corrected_words[verb_index] += "ோம்"  # Add plural suffix if not present

    if errors:
        corrected_sentence = " ".join(corrected_words)
        return {"status": "errors", "details": errors, "corrected_sentence": corrected_sentence}
    else:
        return {"status": "correct", "details": "The sentence is grammatically correct."}


In [21]:
sentence = "பள்ளிக்கு நாங்கள் செல்வேன்"  # Incorrect Tamil sentence
result = check_and_correct_grammar(sentence)

if result["status"] == "correct":
    print(result["details"])
else:
    print("Grammar Errors Found:")
    for error in result["details"]:
        print(f"- {error}")
    if "corrected_sentence" in result:
        print(f"Corrected Sentence: {result['corrected_sentence']}")


Grammar Errors Found:
- Error: The sentence should follow Subject-Object-Verb (SOV) order.
- Error: Plural pronoun should match plural verb form.
Corrected Sentence: நாங்கள் பள்ளிக்கு செல்வோம்
